In [31]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [32]:
# For Google Colab use commands installing packages
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

# Install PICOS and CVXOPT in Google Colab
if IN_COLAB:
    !pip install -q picos
    !pip install -q cvxopt

In [33]:
import picos as pic
import cvxopt as cvx
import Choi as helper

In [34]:
print('Solvers supported on this installation of PICOS:', pic.solvers.all_solvers().keys())

Solvers supported on this installation of PICOS: dict_keys(['cplex', 'cvxopt', 'ecos', 'glpk', 'gurobi', 'mosek', 'mskfsn', 'osqp', 'scip', 'smcp'])


In [35]:
print('Solvers available to PICOS on this machine :', pic.solvers.available_solvers())

Solvers available to PICOS on this machine : ['cvxopt', 'ecos', 'osqp']


In [36]:
def PositiveSemidefiniteOperator(d):
    mt = np.random.rand(d,d) + 1j*np.random.randn(d,d) #A = random C^(d) matrix
    mt = np.dot(mt,mt.conj().T) # A := A . (A*)^T
    return mt/np.trace(mt) # sig := A / tr(A), this is a positive semidefinite matrix

We denote by $\mathcal{P}(\mathcal{H})$ the set of positive semidefinite operators acting on a Hilbert Space $\mathcal{H}$.

Let $\mathcal{N}: A \rightarrow B$ be a quantum channel with Choi operator $\tau_{AB}^\cal{N}$. 

Then
  
  \begin{align}
    \begin{aligned}
  \mathcal{Q}(\cal{N}) \leq \log \varGamma (\cal{N})
    \end{aligned}
   \end{align}
   
Where $\varGamma (\cal{N})$ is given by


  \begin{align}
    \begin{aligned}
    \text{minimize: }\mu \\ \text{subject to: } & \mu \in \mathbb{R} \\ & V_{AB}, Y_{AB} \in \mathcal{P}(\mathcal{H}_A \otimes \mathcal{H}_B) \\ & (V_{AB} - Y_{AB})^{T_B} \geq \tau_{AB}^\cal{N} \\ & V_A + Y_A \leq \mu {\mathbb{I}}_A
    \end{aligned}
   \end{align}

We solve this semidefinite program using PICOS.

In [37]:
choi_mat = np.array([[1,0,0,0],[0,0.5,1,0],[0,1,0.5,0],[0,0,0,0.5]])

In [79]:
import cvxpy as cp
import numpy as np


Vab = cp.Variable((4,4), symmetric=True)
Yab = cp.Variable((4,4), symmetric=True)
mu = cp.Variable()
iMat = np.array([[1,0],[0,1]])

Va = cp.partial_trace(Vab, (2,2), 1)
Ya = cp.partial_trace(Yab, (2,2), 1)

constraints = [Vab >> 0]
constraints += [Yab >> 0]

constraints += [cp.partial_transpose(Vab - Yab, (2,2), 1) >> choi_mat]

constraints += [Va + Ya << mu * iMat]

prob = cp.Problem(cp.Minimize(mu),
                  constraints)
prob.solve()

print(log2(prob.value))

0.5849650140102338
